В данном ноутбуке будет продемонстрировано решение одной из задач сентиментального анализа - **Предсказание эмоциональной окраски отзыва**. Мы разберём с вами загрузку данных из четырёх датасетов: 
1. **Amazon** (UCI)
2. **IMDB** (UCI)
3. **Yelp** (UCI)
4. **Amazon reviews** (Kaggle)<br>

Удалим стоп слова в тексте, выделим основы слов. Далее мы натренируем несколько классификаторов - в том числе и многослойную нейронную сеть на **PyTorch**, сравним несколько метрик (accuracy, confusion_matrix, roc_auc), построим графики и выгрузим нашу модель в Docker. Приступим

### 0. Импортируем библиотеки

In [97]:
import numpy as np
import pandas as pd
import sklearn.metrics
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from lightgbm import LGBMClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [2]:
warnings.filterwarnings("ignore")
%matplotlib inline
plt.style.use("ggplot")
plt.rcParams["figure.figsize"] = (10, 10)

### 1. Загрузим данные

Датасеты, с которыми мы будем работать - были взяты из следующих источников:
1. <a href = "https://www.kaggle.com/sid321axn/amazon-alexa-reviews">Amazon Alexa Reviews | Kaggle</a>
2. <a href = "https://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences">Sentiment Labelled Sentences Data Set | UCI </a>

Датасеты имеют разные форматы. Тот, что с Kaggle - tsv, с UCI - txt. Создадим функции, который будет вытаскивать необходимые для нас данные с обоих форматов

Загрузка таблицы/таблиц формата csv/tsv:

In [55]:
def TsvLoader(paths, delim, text_attribute, rate_attribute):
    dictionary_data = {"data": [], "target": []} # Сохранять данные мы будем в словарь, можно и любую другую структуру данных,
                                                 # мне лично удобнее работать со словарём
        
    for path in paths:
        data = pd.read_csv(path, sep = delim)
        data = data[[text_attribute, rate_attribute]] # Нам нужны только колонки с текстом и оценками
        data[rate_attribute] = data[rate_attribute].astype("int32")
        
        # Очищаем данные от пунктуации и цифр, после загружаем в словарь
        for index, row in data.iterrows():
            line = ""
            for character in row[text_attribute]:
                if character.isalnum() or character == " ":
                    line += character
                    
            dictionary_data["data"].append(line.lower())
            dictionary_data["target"].append(row[rate_attribute])
        
    return dictionary_data

Загрузка данных из формата txt:

In [47]:
def TxtLoader(paths):
    dictionary_data = {"data": [], "target": []}
    
    # Загружаем текст в словарь, попутно очищая от знаков препинания, цифр и приводя к нижнему регистру
    for path in paths:
        f = open(path, "r", encoding = "UTF-8")

        for text in f.readlines():
            line = ''
    
            for character in text[:-2]:
                if character.isalnum() or character == " ":
                    line += character
        
            dictionary_data["data"].append(line.lower()) # Закидываем в data нашу строку
            dictionary_data["target"].append(int(text[-2])) # Закидаываем в target положительный или отрицательный отзыв
        f.close()
    
    return dictionary_data

После загрузки данных, из двух функций мы получим два словаря с данными, нужно написать функцию, которая будет их объединять в один общий словарь:

In [71]:
def concatenateDict(dictionaris):
    extended_dict = {"data": [], "target": []}
    
    for dictionary in dictionaris:
        extended_dict["data"].extend(dictionary["data"])
        extended_dict["target"].extend(dictionary["target"])
        
    return extended_dict

Загружаем данные:

In [56]:
path = [r"C:\Users\Sergei\Desktop\Downloads\datasets\amazon_alexa.tsv"]

tsv_dictionary = TsvLoader(path, "\t", "verified_reviews", "feedback")

In [58]:
path = [r"C:\Users\Sergei\Desktop\Downloads\datasets\amazon_cells_labelled.txt", 
        r"C:\Users\Sergei\Desktop\Downloads\datasets\imdb_labelled.txt",
        r"C:\Users\Sergei\Desktop\Downloads\datasets\yelp_labelled.txt"]

txt_dictionary = TxtLoader(path)

In [59]:
extended_data = concatenateDict([tsv_dictionary, txt_dictionary])

Проверим, совпадают ли размерности начальных словарей и итогового:

In [60]:
len(tsv_dictionary["data"]), len(txt_dictionary["data"]), len(extended_data["data"])

(3150, 3000, 6150)

In [61]:
len(tsv_dictionary["target"]), len(txt_dictionary["target"]), len(extended_data["target"])

(3150, 3000, 6150)

Всё совпадает! Приступим к предобработке данных

### 2. Предобработка данных

Перед разбиением данных на тренировочный и тестовый наборы - нам необходимо предобработать наши тексты. При загрузке наших данных мы уже привели слова к нижнему регистру и удалили знаки пунктуации и цифры. Осталось удалить стоп-слова, которые не несут смысловой нагрузки в тексте (артикли, предлоги и так далее), затем нам необходимо выделить основы слов.  

In [65]:
# Удаляем ненужные словари
del tsv_dictionary 
del txt_dictionary

In [67]:
def clearData(dictionary_data):
    english_stop_words = stopwords.words("english") # Перед использованием, нужно выполнить команду nltk.download('stopwords')
    
    for i, text in enumerate(dictionary_data["data"]):
        line = ""
        splitted = text.split()
        tokenized = [word for word in splitted if word not in english_stop_words]
        
        # Выделяем основы слов
        porter = PorterStemmer()
        cleared = [porter.stem(word) for word in tokenized]
                   
        for word in cleared:
            line += (word + " ")
        
        dictionary_data["data"][i] = line
        
    return dictionary_data

In [68]:
clear_dict = clearData(extended_data)

### 3. Подготавливаем данные

Осталось разбить словарь на тренировочный и тестовый наборы

In [69]:
def train_test_split(data, train_size = 0.77):
    size = int(len(data["data"]) * 0.7)
    
    train_x = data["data"][:size].copy()
    test_x = data["data"][size:].copy()
    train_y = data["target"][:size].copy() 
    test_y = data["target"][size:].copy()
    
    return train_x, test_x, train_y, test_y

In [70]:
train_X, test_X, train_y, test_y = train_test_split(clear_dict, train_size = 0.6)

### 4. Тренировка моделей

In [91]:
names = ["Logit", "SVC", "LGBM"]
classifiers = [
    LogisticRegressionCV(cv = 5),
    SVC(),
    LGBMClassifier()
]

In [95]:
for name, classifier in zip(names, classifiers):
    print("-" * 50 + "[" + name + "]" + "-" * 50)
    vectorized_X = TfidfVectorizer().fit_transform(train_X)
    scores = cross_val_score(classifier, vectorized_X, train_y, cv = 5, scoring = )
    print("Mean accuracy", scores.mean())
    print("Std accuracy", scores.std())

--------------------------------------------------[Logit]--------------------------------------------------
Mean Accuracy 0.8552845528455284
--------------------------------------------------[SVC]--------------------------------------------------
Mean Accuracy 0.8329849012775842
--------------------------------------------------[LGBM]--------------------------------------------------
Mean Accuracy 0.8357723577235772
